In [ ]:
!pip install tensorflow matplotlib


In [129]:
# 必要なライブラリのインポート
import os
import numpy as np
from PIL import Image
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

# データセットフォルダのパス
dataset_path = '/content/drive/MyDrive/'

dataset_path_Lineart = os.path.join(dataset_path, 'Lineart')
dataset_path_Color = os.path.join(dataset_path, 'Color')

# 画像ファイルをリストで取得
image_files_Lineart = os.listdir(dataset_path_Lineart)
image_files_Color = os.listdir(dataset_path_Color)

# テスト用の画像枚数を指定
num_test_images = 10  # 例えば10枚をテストする場合

# テスト用にランダムに画像ファイルを選択
selected_files = random.sample([file for file in image_files_Lineart if file in image_files_Color], num_test_images)


# 白黒画像とカラー画像のペアをロード
def load_image_pair(filename):
    Lineart_image_path = os.path.join(dataset_path_Lineart, filename)  # 白黒画像のパス
    Color_image_path = os.path.join(dataset_path_Color, filename)  # カラー画像のパス

    Lineart_image = load_image(Lineart_image_path)  # 白黒画像をロード
    Color_image = load_image(Color_image_path)  # カラー画像をロード

    return Lineart_image, Color_image

# 選択した画像ペアをロード
image_pairs = [load_image_pair(file) for file in selected_files]

"""
import matplotlib.pyplot as plt

# 画像ペアを表示する関数
def display_image_pairs(image_pairs):
    for i, (lineart, color) in enumerate(image_pairs):
        plt.figure(figsize=(10, 5))

        # 白黒画像を表示
        plt.subplot(1, 2, 1)
        plt.imshow(lineart, cmap='gray')
        plt.title('Lineart')
        plt.axis('off')

        # カラー画像を表示
        plt.subplot(1, 2, 2)
        plt.imshow(color)
        plt.title('Color')
        plt.axis('off')

        plt.show()

# 画像ペアを表示
display_image_pairs(image_pairs)
"""






"\nimport matplotlib.pyplot as plt\n\n# 画像ペアを表示する関数\ndef display_image_pairs(image_pairs):\n    for i, (lineart, color) in enumerate(image_pairs):\n        plt.figure(figsize=(10, 5))\n\n        # 白黒画像を表示\n        plt.subplot(1, 2, 1)\n        plt.imshow(lineart, cmap='gray')\n        plt.title('Lineart')\n        plt.axis('off')\n\n        # カラー画像を表示\n        plt.subplot(1, 2, 2)\n        plt.imshow(color)\n        plt.title('Color')\n        plt.axis('off')\n\n        plt.show()\n\n# 画像ペアを表示\ndisplay_image_pairs(image_pairs)\n"

In [163]:
class ImageDataset(Dataset):
    def __init__(self, image_pairs):
        self.image_pairs = image_pairs

    def __len__(self):
        return len(self.image_pairs)

    # 画像のロード関数
    def load_image(self, image_path, mode='RGB'):
        # ファイルパスが与えられた場合は画像を読み込む
        if isinstance(image_path, str):  # 画像パスの場合
            image = Image.open(image_path).resize((256, 256))  # リサイズ
            if mode == 'L':  # グレースケールに変換
                image = image.convert('L')  # グレースケール変換
            else:
                image = image.convert('RGB')  # RGB画像として読み込み
            image = np.array(image) / 255.0  # 0-1の範囲に正規化
        else:  # すでに numpy.ndarray が与えられた場合
            image = image_path
        return image

    def __getitem__(self, idx):
        lineart_image_path, color_image_path = self.image_pairs[idx]

        # 白黒のlineart画像はグレースケールとしてロード
        lineart_image = self.load_image(lineart_image_path, mode='L')

        # カラー画像はRGBとしてロード
        color_image = self.load_image(color_image_path, mode='RGB')

        # データをテンソルに変換
        # lineart_imageはグレースケールなので、unsqueeze(0)でチャンネル次元を追加
        return torch.tensor(lineart_image, dtype=torch.float32).unsqueeze(0), torch.tensor(color_image, dtype=torch.float32)

# データセットの作成
dataset = ImageDataset(image_pairs)
batch_size = 16  # 任意のバッチサイズ
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [164]:
# Generatorの定義
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # エンコーダー部分の層
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1),  # 入力: 白黒画像
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
        )
        # デコーダー部分の層
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),  # 出力: カラー画像
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Discriminatorの定義
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),  # 入力: 白黒画像 + カラー画像
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 1, kernel_size=4, stride=1, padding=1),)
    def forward(self, x):
        return self.model(x)


In [165]:
# モデルの初期化
generator = Generator()
discriminator = Discriminator()

# オプティマイザーの設定
criterion = nn.BCELoss()  # バイナリ交差エントロピー損失
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))


In [167]:
# 学習ループ
num_epochs = 200  # エポック数
for epoch in range(num_epochs):
    for i, (lineart, color) in enumerate(data_loader):
        # lineartが5次元なら、次元を削除 (不要な次元の場合のみ)
        if lineart.dim() == 5:
            lineart = lineart.squeeze(1)  # [B, 1, 256, 256, 3] -> [B, 256, 256, 3]

        if color.dim() == 5:
            color = color.squeeze(1)      # カラー画像も同様に処理

        if color.dim() == 4:  # [B, 256, 256, 3] の場合
            color = color.permute(0, 3, 1, 2)      # カラー画像も同じ処理

        real_images = color
        real_labels = torch.ones(real_images.size(0), 1)  # 本物ラベル
        fake_labels = torch.zeros(real_images.size(0), 1)  # 偽物ラベル

        # Generatorの学習
        optimizer_G.zero_grad()
        print(lineart.shape)

        generated_images = generator(lineart)
        output = discriminator(torch.cat((lineart, generated_images), 1))
        g_loss = criterion(output, real_labels)  # 損失の計算
        g_loss.backward()
        optimizer_G.step()

        # Discriminatorの学習
        optimizer_D.zero_grad()
        real_output = discriminator(torch.cat((lineart, real_images), 1))
        d_loss_real = criterion(real_output, real_labels)

        fake_output = discriminator(torch.cat((lineart, generated_images.detach()), 1))
        d_loss_fake = criterion(fake_output, fake_labels)

        d_loss = d_loss_real + d_loss_fake  # 総合的な損失
        d_loss.backward()
        optimizer_D.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss D: {d_loss.item()}, Loss G: {g_loss.item()}')

torch.Size([10, 256, 256, 3])


RuntimeError: Given groups=1, weight of size [64, 1, 4, 4], expected input[10, 256, 256, 3] to have 1 channels, but got 256 channels instead